In [4]:
import numpy as np
import sys
import csv
import pandas as pd



In [ ]:
## This script will run with coverage data for each chromosome separately.
## The files should have the form: 
### FileName + chrX + .txt

In [35]:
## what are the variables:
### -- inFile: is the path and filename of the file from which we want to cut the columns 
### -- windowSize: in what window should the coverage average be evaluated? (I am here not letting windows overlap,
###    because that would defeat the cause of making the file size smaller)
### -- out: path and name of the output sample

#InFile = sys.argv[1]
#out = sys.argv[2]

#print ("This is the filename:", CutFile)
#print ("These are the columns:", columns)
windowSize = 10000
InPath = "/home/isabel/Documents/postDoc_Amsterdam/1_EvolWormJourney/1_Genomics/2_EvolGenomics/1_Dev_PipelineCelegans/Coverage/fromMapping/"
InFile="Gent_bb_1_1_CE_mapped_compl_cov_"
suffix = ".txt"

chromosomes=["NC_003279.8","NC_003280.10","NC_003281.10","NC_003282.8","NC_003283.11","NC_003284.9","NC_001328.1"]
chromosomes_length = [15072434, 15279421, 13783801, 17493829, 20924180, 17718942, 13794]
chromosomes_end = [1.50750e+07, 3.03500e+07, 4.41350e+07, 6.16300e+07, 8.25550e+07, 1.00270e+08, 1.00285e+08]
chr_shortNames = ["chr1","chr2","chr3","chr4","chr5","chr6","chr7"]


In [ ]:

df_sldCov = pd.DataFrame({
    "ID":[],
    "chr":[],
    "pos":[],
    "cov_sldWinMean":[]})


for i in range(len(chr_shortNames)):
    
    df_pre=pd.read_csv(InPath + InFile + chr_shortNames[i] + suffix,header=None, delimiter="\t", index_col=False, na_values="na")
    df_pre.columns=["chr",'pos','coverage']

    # test some things:
    if (df_pre.chr.values[0] == df_pre.chr.values[-1]) & (df_pre.chr.values[0] == chromosomes[i]) & (df_pre.chr.values[-1] == chromosomes_length[i]):
        print(df_pre.chr[0])
        continue

    # for each chromosome, get the positions at which to evaulate the coverage
    # always evaluate at the middle of the window 
    # example, windowSize=10000, evaluate at 5000 from 0 to 10000, at 15000 from 10001 to 20000 etc.
    loopNum = round(chromosomes_length[i]/windowSize)
    loopPos = np.round(np.arange(windowSize/2, (loopNum)*windowSize + windowSize/2, windowSize))
    posHere = df_pre.pos.values
    covHere = df_pre.coverage.values

    # initiate the data
    cov_sldWinMean = np.zeros(len(loopPos))

    # run the loop over all positions but not the last
    for j in range(len(loopPos)):
        if (loopPos[j] == loopPos[-1]) & (chromosomes_length[i] < (loopPos[j] + windowSize/2)):
            print(i)
            print(j)
            print("True")
            lastWinStart = int((loopPos[j]-windowSize/2))
            cov_sldWinMean[j] = np.nanmean(covHere[lastWinStart:posHere[-1]])
        else:    
            maskHere = posHere[(posHere > (loopPos[j] - windowSize/2)) & (posHere <= (loopPos[j] + windowSize/2))]
            cov_sldWinMean[j] = np.nanmean(covHere[maskHere])
    print(cov_sldWinMean)
    
    IDs = list(map(str.__add__, np.array(df_pre['chr'].values, dtype=str), np.array(np.array(loopPos, dtype = int), dtype=str)))
    df_here = pd.DataFrame({
        "ID": IDs,
        "chr": np.transpose(np.repeat(chromosomes[0],len(loopPos))),
        "pos": loopPos,
        "cov_sldWinMean": cov_sldWinMean})

    #pd.concat([df_sldCov,df_here])
    

[ 192.8008  173.6985  191.0269 ...  241.044   235.4987 3984.5619]
1
1527
True
[212.8057     178.7134     213.3741     ... 203.491      203.4608
 207.81275873]
[139.9072 192.0152 193.4631 ... 227.3658 241.0931 228.0457]
[ 29.4246  25.943   33.0643 ... 227.2083 213.6718 203.8156]


In [175]:
IDs = list(map(str.__add__, np.array(df_pre['chr'].values, dtype=str), np.array(np.array(loopPos, dtype = int), dtype=str)))
IDs

['NC_003279.85000',
 'NC_003279.815000',
 'NC_003279.825000',
 'NC_003279.835000',
 'NC_003279.845000',
 'NC_003279.855000',
 'NC_003279.865000',
 'NC_003279.875000',
 'NC_003279.885000',
 'NC_003279.895000',
 'NC_003279.8105000',
 'NC_003279.8115000',
 'NC_003279.8125000',
 'NC_003279.8135000',
 'NC_003279.8145000',
 'NC_003279.8155000',
 'NC_003279.8165000',
 'NC_003279.8175000',
 'NC_003279.8185000',
 'NC_003279.8195000',
 'NC_003279.8205000',
 'NC_003279.8215000',
 'NC_003279.8225000',
 'NC_003279.8235000',
 'NC_003279.8245000',
 'NC_003279.8255000',
 'NC_003279.8265000',
 'NC_003279.8275000',
 'NC_003279.8285000',
 'NC_003279.8295000',
 'NC_003279.8305000',
 'NC_003279.8315000',
 'NC_003279.8325000',
 'NC_003279.8335000',
 'NC_003279.8345000',
 'NC_003279.8355000',
 'NC_003279.8365000',
 'NC_003279.8375000',
 'NC_003279.8385000',
 'NC_003279.8395000',
 'NC_003279.8405000',
 'NC_003279.8415000',
 'NC_003279.8425000',
 'NC_003279.8435000',
 'NC_003279.8445000',
 'NC_003279.8455000',

In [169]:
str(cov_sldWinMean) + str(loopPos)

'[ 192.8008  173.6985  191.0269 ...  241.044   235.4987 3984.5619][5.0000e+03 1.5000e+04 2.5000e+04 ... 1.5045e+07 1.5055e+07 1.5065e+07]'

## What you need:
* 

In [141]:
a = np.repeat("A",5)
a[2]

'A'

In [99]:
for i in range(len(loopPos)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

array([5.0000e+03, 1.5000e+04, 2.5000e+04, ..., 1.5045e+07, 1.5055e+07,
       1.5065e+07])

In [57]:
windowSize/2

5000.0